# Kaggle Datasets Import

[![Jupyter Notebook](https://img.shields.io/badge/jupyter-%23FA0F00.svg?style=for-the-badge&logo=jupyter&logoColor=white)](https://github.com/openvinotoolkit/datumaro/blob/develop/notebooks/20_kaggle_data_import.ipynb)

[Kaggle](https://www.kaggle.com/) is one of the largest machine learning communities that publishes datasets, models, and code. Moreover, Kaggle also hosts competitions.

Specifically, Datumaro focuses on Kaggle computer vision datasets, comprising more than 28,000 datasets across various domains such as medical, nature, road scenes, and people.

This notebook demonstrates how Datumaro imports Kaggle datasets from scratch, covering the entire process from data preparation to investigation and import.

### Download a Kaggle dataset
Thanks to the Kaggle CLI, we are able to show the list of Kaggle datasets according to the tag `Image` as below.

In [2]:
!pip install kaggle
!kaggle datasets list --tags Image --sort-by votes

ref                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
paultimothymooney/chest-xray-pneumonia          Chest X-Ray Images (Pneumonia)                       2GB  2018-03-24 19:41:59         267348       6141  0.75             
moltean/fruits                                  Fruits-360                                           1GB  2021-09-12 19:19:48         135209       2833  0.875            
zalando-research/fashionmnist                   Fashion MNIST                                       69MB  2017-12-07 00:54:20         177934       2646  0.85294116       
muratkokludataset/date-fruit-datasets           Date Fruit Datasets                                408KB  2022-04-03 09:25:39          21425     

Also, we can directly download a dataset from the above `ref` field.

In this notebook, we choose [alxmamaev/flowers-recognition](https://www.kaggle.com/datasets/alxmamaev/flowers-recognition) dataset as below.

In [2]:
!kaggle datasets download alxmamaev/flowers-recognition --unzip --path ./flowers-recognition

 99%|███████████████████████████████████████▋| 223M/225M [00:11<00:00, 23.9MB/s]
100%|████████████████████████████████████████| 225M/225M [00:11<00:00, 20.5MB/s]


Since Kaggle doesn't require any specific format for datasets, we should detect available formats from Datumaro `detect` method or explore a dataset directory structure by hand.

### Case 1: Detect Data Format & Import Dataset with Format

In [8]:
import datumaro as dm

dataset_dir = "./flowers-recognition/flowers"
formats = dm.Dataset.detect(dataset_dir)
print(f"Detected data format is '{formats}'")

dataset = dm.Dataset.import_from(dataset_dir, formats)
print(dataset)

Detected data format is 'imagenet'
Dataset
	size=4317
	source_path=./flowers-recognition/flowers
	media_type=<class 'datumaro.components.media.Image'>
	annotated_items_count=4317
	annotations_count=4317
subsets
	default: # of items=4317, # of annotated items=4317, # of annotations=4317, annotation types=['label']
infos
	categories
	label: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']



### Case 2: Import Dataset with More Arguments for Kaggle Importers

From Datumaro 2.0, we have introduced some Kaggle importers such as `kaggle_voc`, `kaggle_yolo`, `kaggle_image_csv`, `kaggle_image_txt`, and `kaggle_image_mask`.

Some of them are variants of the public format by loosening strict rules of the original format, while some of them require user assistance to feed rules for reading an annotation file.

For instances, `kaggle_voc` is a variant of `Pasal-VOC` format, which contain `xml` files for each images, but `kaggle_voc` doesn't restrict to have a given directory structure.

Instead, `kaggle_voc` requires to have both image path and annotation path as below.

We select [andrewmvd/face-mask-detection](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection) for example.

In [18]:
!kaggle datasets download andrewmvd/face-mask-detection --unzip --path ./face-mask-detection
!tree ./face-mask-detection -L 1

100%|███████████████████████████████████████▉| 397M/398M [00:20<00:00, 21.0MB/s]
100%|████████████████████████████████████████| 398M/398M [00:20<00:00, 19.9MB/s]
./face-mask-detection
├── annotations
└── images

2 directories, 0 files


In [19]:
dataset_dir = "./face-mask-detection/images"
ann_path = "./face-mask-detection/annotations"

dataset = dm.Dataset.import_from(path=dataset_dir, format="kaggle_voc", ann_path=ann_path)
print(dataset)

Dataset
	size=853
	source_path=./face-mask-detection/images
	media_type=<class 'datumaro.components.media.Image'>
	annotated_items_count=853
	annotations_count=4072
subsets
	default: # of items=853, # of annotated items=853, # of annotations=4072, annotation types=['bbox']
infos
	categories
	label: ['with_mask', 'without_mask', 'mask_weared_incorrect']



On the other hands, we have found that there are some patterns between Kaggle datasets.

One of the patterns is existence of annotation file with `csv` or `txt` for image path, labels, bounding box coordinates, etc.

And hence, we provide two importers, e.g., `kaggle_image_csv` and `kaggle_image_txt`, for coping with these datasets.

We here examine the 7-th most voted dataset [kmader/skin-cancer-mnist-ham10000](https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000), more than 106K times of downloaded and 1720 times voted.

In [15]:
!kaggle datasets download kmader/skin-cancer-mnist-ham10000 --unzip --path ./skin-cancer-mnist-ham10000
!tree ./skin-cancer-mnist-ham10000 -L 1

100%|██████████████████████████████████████| 5.20G/5.20G [04:21<00:00, 22.2MB/s]
100%|██████████████████████████████████████| 5.20G/5.20G [04:21<00:00, 21.3MB/s]
./skin-cancer-mnist-ham10000
├── ham10000_images_part_1
├── HAM10000_images_part_1
├── ham10000_images_part_2
├── HAM10000_images_part_2
├── HAM10000_metadata.csv
├── hmnist_28_28_L.csv
├── hmnist_28_28_RGB.csv
├── hmnist_8_8_L.csv
└── hmnist_8_8_RGB.csv

4 directories, 5 files


In [17]:
!head -10 ./skin-cancer-mnist-ham10000/HAM10000_metadata.csv

lesion_id,image_id,dx,dx_type,age,sex,localization
HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back


From the above `csv` file, we are going to parse `image_id` for image path and `dx` for label information.

They will be fed into importer as `columns`.

In [14]:
dataset_dir = "./skin-cancer-mnist-ham10000/ham10000_images_part_1"
ann_file = "./skin-cancer-mnist-ham10000/HAM10000_metadata.csv"
columns = {"media": "image_id", "label": "dx"}

dataset = dm.Dataset.import_from(
    path=dataset_dir, format="kaggle_image_csv", ann_file=ann_file, columns=columns
)
print(dataset)

/home/wonjulee/datumaro/src/datumaro/plugins/data_formats/kaggle/base.py:83: UserWarning: './skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0031633.dib' is not existed in the directory, so we skip to create an dataset item according to 0     HAM_0001466
1    ISIC_0031633
2             bkl
3           histo
4            75.0
5            male
6             ear
Name: 5, dtype: object.
  warnings.warn(
/home/wonjulee/datumaro/src/datumaro/plugins/data_formats/kaggle/base.py:83: UserWarning: './skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0029396.dib' is not existed in the directory, so we skip to create an dataset item according to 0     HAM_0004234
1    ISIC_0029396
2             bkl
3           histo
4            85.0
5          female
6           chest
Name: 12, dtype: object.
  warnings.warn(
/home/wonjulee/datumaro/src/datumaro/plugins/data_formats/kaggle/base.py:83: UserWarning: './skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0032417.dib' is not existed i

Dataset
	size=5000
	source_path=./skin-cancer-mnist-ham10000/ham10000_images_part_1
	media_type=<class 'datumaro.components.media.Image'>
	annotated_items_count=5000
	annotations_count=5000
subsets
	default: # of items=5000, # of annotated items=5000, # of annotations=5000, annotation types=['label']
infos
	categories
	label: ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']



### Concluding Remarks

We are still increasing the coverage rate of Kaggle datasets with the aid of Datumaro importers.

Currently, the coverage rate is 80% for top-100 most voted `Image` datasets.

Please enjoy the arguments for importing various Kaggle datasets below.

In [4]:
datasets = [
    {
        "path": "./paultimothymooney/chest-xray-pneumonia/chest_xray",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./moltean/fruits/fruits-360_dataset/fruits-360/Training", "format": "imagenet"},
    {"path": "./zalando-research/fashionmnist", "format": "mnist_csv"},
    {"path": "./alxmamaev/flowers-recognition/flowers", "format": "imagenet"},
    {"path": "./muratkokludataset/rice-image-dataset/Rice_Image_Dataset", "format": "imagenet"},
    {
        "path": "./andrewmvd/face-mask-detection/images",
        "format": "kaggle_voc",
        "ann_path": "./andrewmvd/face-mask-detection/annotations",
    },
    {
        "path": "./kmader/skin-cancer-mnist-ham10000/ham10000_images_part_1",
        "format": "kaggle_image_csv",
        "ann_file": "./kmader/skin-cancer-mnist-ham10000/HAM10000_metadata.csv",
        "columns": {"media": "image_id", "label": "dx"},
    },
    {
        "path": "./muratkokludataset/pistachio-image-dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset",
        "format": "imagenet",
    },
    {
        "path": "./muratkokludataset/durum-wheat-dataset/Durum_Wheat_Dataset/Dataset2-Durum Wheat Video Images",
        "format": "imagenet",
    },
    {"path": "./puneet6060/intel-image-classification/seg_train/seg_train/", "format": "imagenet"},
    {
        "path": "./gpiosenka/100-bird-species",
        "format": "kaggle_image_csv",
        "ann_file": "./gpiosenka/100-bird-species/birds.csv",
        "columns": {"media": "filepaths", "label": "labels"},
    },
    {
        "path": "./meowmeowmeowmeowmeow/gtsrb-german-traffic-sign",
        "format": "kaggle_image_csv",
        "ann_file": "./meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/Train.csv",
        "columns": {
            "media": "Path",
            "bbox": {
                "x1": "Roi.X1",
                "y1": "Roi.Y1",
                "x2": "Roi.X2",
                "y2": "Roi.Y2",
            },
            "label": "ClassId",
        },
    },
    {"path": "./ikarus777/best-artworks-of-all-time/images/images", "format": "imagenet"},
    {
        "path": "./crowww/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Shrimp/Shrimp",
        "format": "kaggle_image_mask",
        "mask_path": "./crowww/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Shrimp/Shrimp GT",
    },
    {"path": "./paultimothymooney/breast-histopathology-images/10253/", "format": "imagenet"},
    {
        "path": "./tawsifurrahman/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images",
        "format": "kaggle_image_mask",
        "mask_path": "./tawsifurrahman/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/masks",
    },
    {
        "path": "./grassknoted/asl-alphabet/asl_alphabet_train/asl_alphabet_train",
        "format": "imagenet",
    },
    {
        "path": "./vipoooool/new-plant-diseases-dataset/new plant diseases dataset(augmented)/"
        + "New Plant Diseases Dataset(Augmented)",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./paultimothymooney/blood-cells/dataset-master/dataset-master/JPEGImages/",
        "format": "kaggle_voc",
        "ann_path": "./paultimothymooney/blood-cells/dataset-master/dataset-master/Annotations",
    },
    {"path": "./alessiocorrado99/animals10/raw-img", "format": "imagenet"},
    {"path": "./oddrationale/mnist-in-csv", "format": "mnist_csv"},
    {
        "path": "./sachinpatel21/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data",
        "format": "tabular",
    },
    {"path": "./gti-upm/leapgestrecog/leapGestRecog/00", "format": "imagenet"},
    {
        "path": "./paramaggarwal/fashion-product-images-dataset/fashion-dataset/images",
        "format": "kaggle_image_csv",
        "ann_file": "./paramaggarwal/fashion-product-images-dataset/fashion-dataset/styles.csv",
        "columns": {"media": "id", "label": "masterCategory"},
    },
    {
        "path": "./alexattia/the-simpsons-characters-dataset",
        "format": "kaggle_image_txt",
        "ann_file": "./alexattia/the-simpsons-characters-dataset/annotation.txt",
        "columns": {
            "media": 0,
            "bbox": {
                "x1": 1,
                "y1": 2,
                "width": 3,
                "height": 4,
            },
            "label": 5,
        },
    },
    {
        "path": "./paultimothymooney/kermany2018/oct2017/OCT2017 ",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./slothkong/10-monkey-species/training/training", "format": "imagenet"},
    {
        "path": "./tanlikesmath/diabetic-retinopathy-resized/resized_train/resized_train",
        "format": "kaggle_image_csv",
        "ann_file": "./tanlikesmath/diabetic-retinopathy-resized/trainLabels.csv",
        "columns": {"media": "image", "label": "level"},
    },
    {"path": "./jboysen/mri-and-alzheimers", "format": "tabular"},
    {
        "path": "./asdasdasasdas/garbage-classification/garbage classification/Garbage classification",
        "format": "imagenet",
    },
    {"path": "./prasunroy/natural-images/natural_images", "format": "imagenet"},
    {"path": "./shaunthesheep/microsoft-catsvsdogs-dataset/PetImages", "format": "imagenet"},
    {
        "path": "./kmader/rsna-bone-age/boneage-training-dataset/boneage-training-dataset",
        "format": "kaggle_image_csv",
        "ann_file": "./kmader/rsna-bone-age/boneage-training-dataset.csv",
        "columns": {"media": "id", "label": "boneage"},
    },
    {
        "path": "./techsash/waste-classification-data/dataset/DATASET",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./landlord/handwriting-recognition/train_v2",
        "format": "kaggle_image_csv",
        "ann_file": "./landlord/handwriting-recognition/written_name_train_v2.csv",
        "columns": {"media": "FILENAME", "label": "IDENTITY"},
    },
    {
        "path": "./andrewmvd/ocular-disease-recognition-odir5k/ODIR-5K/Training Images/",
        "format": "kaggle_image_csv",
        "ann_file": "./andrewmvd/ocular-disease-recognition-odir5k/full_df.csv",
        "columns": {"media": "filename", "label": "labels"},
    },
    {
        "path": "./xhlulu/140k-real-and-fake-faces/real_vs_fake/real-vs-fake",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./mbkinaci/fruit-images-for-object-detection/train_zip/train",
        "format": "kaggle_voc",
        "ann_path": "./mbkinaci/fruit-images-for-object-detection/train_zip/train",
    },
    {"path": "./masoudnickparvar/brain-tumor-mri-dataset", "format": "imagenet_with_subset_dirs"},
    {"path": "./fournierp/captcha-version-2-images/samples/samples", "format": "image_dir"},
    {"path": "./chrisfilo/fruit-recognition", "format": "imagenet"},
    {
        "path": "./andrewmvd/hard-hat-detection/images",
        "format": "kaggle_voc",
        "ann_path": "./andrewmvd/hard-hat-detection/annotations",
    },
    {"path": "./nipunarora8/age-gender-and-ethnicity-face-data-csv", "format": "tabular"},
    {
        "path": "./paramaggarwal/fashion-product-images-small/images",
        "format": "kaggle_image_csv",
        "ann_file": "./paramaggarwal/fashion-product-images-small/styles.csv",
        "columns": {"media": "id", "label": "masterCategory"},
    },
    {"path": "./kvpratama/pokemon-images-dataset/pokemon/pokemon", "format": "image_dir"},
    {
        "path": "./ravirajsinh45/real-life-industrial-dataset-of-casting-product/casting_512x512/casting_512x512",
        "format": "imagenet",
    },
    {"path": "./mohamedhanyyy/chest-ctscan-images/Data/", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./thedownhill/art-images-drawings-painting-sculpture-engraving/dataset/dataset_updated",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./ihelon/lego-minifigures-classification",
        "format": "kaggle_image_csv",
        "ann_file": "./ihelon/lego-minifigures-classification/index.csv",
        "columns": {"media": "path", "label": "class_id"},
    },
    {
        "path": "./kmader/finding-lungs-in-ct-data/2d_images/",
        "format": "kaggle_image_mask",
        "mask_path": "./kmader/finding-lungs-in-ct-data/2d_masks/",
    },
    {
        "path": "./aryashah2k/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign",
        "format": "kaggle_image_mask",
        "mask_path": "./aryashah2k/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign",
    },
    {"path": "./ruizgara/socofing/socofing/SOCOFing/Real", "format": "image_dir"},
    {
        "path": "./kritikseth/fruit-and-vegetable-image-recognition",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./jenny18/honey-bee-annotated-images/bee_imgs",
        "format": "kaggle_image_csv",
        "ann_file": "./jenny18/honey-bee-annotated-images/bee_data.csv",
        "columns": {"media": "file", "label": "health"},
    },
    {
        "path": "./iamsouravbanerjee/animal-image-dataset-90-different-animals/animals/animals",
        "format": "imagenet",
    },
    {
        "path": "./nih-chest-xrays/sample/sample/images",
        "format": "kaggle_image_csv",
        "ann_file": "./nih-chest-xrays/sample/sample/sample_labels.csv",
        "columns": {"media": "Image Index", "label": "Finding Labels"},
    },
    {"path": "./andrewmvd/animal-faces/afhq", "format": "imagenet_with_subset_dirs"},
    {"path": "./awsaf49/coco-2017-dataset/coco2017", "format": "mmdet_coco"},
    {
        "path": "./bulentsiyah/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images",
        "format": "kaggle_image_mask",
        "mask_path": "./bulentsiyah/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic",
        "labelmap_file": "./bulentsiyah/semantic-drone-dataset/class_dict_seg.csv",
    },
    {"path": "./plameneduardo/sarscov2-ctscan-dataset", "format": "imagenet"},
    {"path": "./dansbecker/cityscapes-image-pairs/cityscapes_data", "format": "image_dir"},
    {
        "path": "./dasmehdixtr/drone-dataset-uav/dataset_xml_format/dataset_xml_format",
        "format": "kaggle_voc",
        "ann_path": "./dasmehdixtr/drone-dataset-uav/dataset_xml_format/dataset_xml_format",
    },
    {
        "path": "./dasmehdixtr/drone-dataset-uav/drone_dataset_yolo/dataset_txt",
        "format": "kaggle_yolo",
        "ann_path": "./dasmehdixtr/drone-dataset-uav/drone_dataset_yolo/dataset_txt",
    },
    {
        "path": "./minhhuy2810/rice-diseases-image-dataset/LabelledRice/Labelled",
        "format": "imagenet",
    },
    {"path": "./anokas/kuzushiji", "format": "mnist"},
    {
        "path": "./olgabelitskaya/classification-of-handwritten-letters/letters",
        "format": "kaggle_image_csv",
        "ann_file": "./olgabelitskaya/classification-of-handwritten-letters/letters.csv",
        "columns": {"media": "file", "label": "letter"},
    },
    {"path": "./diyer22/retail-product-checkout-dataset/", "format": "imagenet_with_subset_dirs"},
    {"path": "./jangedoo/utkface-new/UTKFace", "format": "image_dir"},
    {"path": "./arnaud58/landscape-pictures", "format": "image_dir"},
    {"path": "./joosthazelzet/lego-brick-images/LEGO brick images v1", "format": "imagenet"},
    {
        "path": "./olgabelitskaya/flower-color-images/flower_images/flower_images",
        "format": "kaggle_image_csv",
        "ann_file": "./olgabelitskaya/flower-color-images/flower_images/flower_images/flower_labels.csv",
        "columns": {"media": "file", "label": "label"},
    },
    {"path": "./ananthu017/emotion-detection-fer", "format": "imagenet_with_subset_dirs"},
    {"path": "./jessicali9530/caltech256/256_ObjectCategories", "format": "imagenet"},
    {
        "path": "./gpiosenka/sports-classification",
        "format": "kaggle_image_csv",
        "ann_file": "./gpiosenka/sports-classification/sports.csv",
        "columns": {"media": "filepaths", "label": "labels"},
    },
    {"path": "./soumikrakshit/anime-faces", "format": "image_dir"},
    {
        "path": "./andrewmvd/leukemia-classification/C-NMC_Leukemia/training_data/fold_0",
        "format": "imagenet",
    },
    {
        "path": "./andrewmvd/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets",
        "format": "imagenet",
    },
    {"path": "./drgfreeman/rockpaperscissors/rps-cv-images", "format": "imagenet"},
    {"path": "./pankrzysiu/cifar10-python/cifar-10-batches-py", "format": "cifar"},
    {"path": "./koryakinp/chess-positions", "format": "image_dir"},
    {"path": "./nancyalaswad90/breast-cancer-dataset", "format": "tabular"},
    {"path": "./clorichel/boat-types-recognition", "format": "imagenet"},
    {"path": "./smeschke/four-shapes/shapes", "format": "imagenet"},
    {
        "path": "./andyczhao/covidx-cxr2/train",
        "format": "kaggle_image_txt",
        "ann_file": "./andyczhao/covidx-cxr2/train.txt",
        "columns": {"media": 1, "label": 2},
    },
    {
        "path": "./nodoubttome/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./vencerlanz09/sea-animals-image-dataste", "format": "imagenet"},
    {
        "path": "./kmkarakaya/logos-bk-kfc-mcdonald-starbucks-subway-none/logos3",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./shubhamgoel27/dermnet", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./nikhilpandey360/chest-xray-masks-and-labels/data/Lung Segmentation/CXR_png",
        "format": "kaggle_image_mask",
        "mask_path": "./nikhilpandey360/chest-xray-masks-and-labels/data/Lung Segmentation",
    },
    {"path": "./vbookshelf/rice-leaf-diseases/rice_leaf_diseases", "format": "imagenet"},
    {
        "path": "./khoongweihao/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./hojjatk/mnist-dataset", "format": "mnist"},
    {"path": "./fatiimaezzahra/famous-iconic-women/output", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./bryanb/abstract-art-gallery/Abstract_gallery/Abstract_gallery",
        "format": "image_dir",
    },
    {
        "path": "./hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images/Shoe vs Sandal vs Boot Dataset",
        "format": "imagenet",
    },
    {"path": "./ifigotin/imagenetmini-1000/imagenet-mini", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./sshikamaru/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./andrewmvd/retinal-disease-classification/Training_Set/Training_Set/Training/",
        "format": "kaggle_image_csv",
        "ann_file": "./andrewmvd/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv",
        "columns": {"media": "ID", "label": "Disease_Risk"},
    },
    {"path": "./tanlikesmath/the-oxfordiiit-pet-dataset/images", "format": "image_dir"},
    {
        "path": "./BengaliAI/numta/training-a",
        "format": "kaggle_image_csv",
        "ann_file": "./BengaliAI/numta/training-a.csv",
        "columns": {"media": "filename", "label": "digit"},
    },
    {"path": "./sayangoswami/reddit-memes-dataset/memes/memes", "format": "image_dir"},
    {"path": "./jehanbhathena/weather-dataset/dataset", "format": "imagenet"},
    {
        "path": "./kumaresanmanickavelu/lyft-udacity-challenge/dataa/dataA/CameraRGB",
        "format": "kaggle_image_mask",
        "mask_path": "./kumaresanmanickavelu/lyft-udacity-challenge/dataa/dataA/CameraSeg",
    },
    {"path": "./fedesoriano/cifar100", "format": "cifar"},
    {
        "path": "./andrewmvd/doom-crossing/animal_crossing/",
        "format": "kaggle_image_csv",
        "ann_file": "./andrewmvd/doom-crossing/animal_crossing_dataset.csv",
        "columns": {"media": "filename", "label": "total_awards_received"},
    },
    {"path": "./cactus3/basicshapes/shapes/", "format": "imagenet"},
    {
        "path": "./agrigorev/clothing-dataset-full/images_compressed",
        "format": "kaggle_image_csv",
        "ann_file": "./agrigorev/clothing-dataset-full/images.csv",
        "columns": {"media": "image", "label": "label"},
    },
    {"path": "./odins0n/ucf-crime-dataset/", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./hgunraj/covidxct/3A_images",
        "format": "kaggle_image_txt",
        "ann_file": "./hgunraj/covidxct/val_COVIDx_CT-3A.txt",
        "columns": {
            "media": 0,
            "bbox": {
                "x1": 2,
                "y1": 3,
                "x2": 4,
                "y2": 5,
            },
            "label": 1,
        },
    },
    {"path": "./mohamedhanyyy/chest-ctscan-images/Data/train", "format": "imagenet"},
    {"path": "./plameneduardo/sarscov2-ctscan-dataset", "format": "imagenet"},
    {
        "path": "./dhruvildave/english-handwritten-characters-dataset/",
        "format": "kaggle_image_csv",
        "ann_file": "./dhruvildave/english-handwritten-characters-dataset/english.csv",
        "columns": {"media": "image", "label": "label"},
    },
    {"path": "./tunguz/deep-solar-dataset", "format": "tabular"},
    {
        "path": "./maysee/mushrooms-classification-common-genuss-images/Mushrooms",
        "format": "imagenet",
    },
    {
        "path": "./sanikamal/horses-or-humans-dataset/horse-or-human/horse-or-human",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./misrakahmed/vegetable-image-dataset/Vegetable Images",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./ambarish/breakhis",
        "format": "kaggle_image_csv",
        "ann_file": "./ambarish/breakhis/Folds.csv",
        "columns": {"media": "filename", "label": "fold"},
    },
    {"path": "./aelchimminut/fruits262/Fruit-262", "format": "imagenet"},
    {"path": "./brsdincer/vehicle-detection-image-set/data", "format": "imagenet"},
    {
        "path": "./valentynsichkar/traffic-signs-dataset-in-yolo-format/ts/ts",
        "format": "kaggle_yolo",
        "ann_path": "./valentynsichkar/traffic-signs-dataset-in-yolo-format/ts/ts",
    },
    {
        "path": "./aalborguniversity/aau-rainsnow",
        "format": "kaggle_coco",
        "ann_file": "./aalborguniversity/aau-rainsnow/aauRainSnow-rgb.json",
    },
    {"path": "./kostastokis/simpsons-faces/cropped", "format": "image_dir"},
    {"path": "./sachinkumar413/alzheimer-mri-dataset/Dataset", "format": "imagenet"},
    {
        "path": "./mistag/arthropod-taxonomy-orders-object-detection-dataset/ArTaxOr",
        "format": "imagenet",
    },
    {"path": "./jessicali9530/stl10/train_images", "format": "image_dir"},
    {
        "path": "./vencerlanz09/pharmaceutical-drugs-and-vitamins-synthetic-images/"
        + "ImageClassesCombinedWithCOCOAnnotations/images_raw",
        "format": "kaggle_coco",
        "ann_file": "./vencerlanz09/pharmaceutical-drugs-and-vitamins-synthetic-images/"
        + "/ImageClassesCombinedWithCOCOAnnotations/coco_instances.json",
    },
    {"path": "./anshtanwar/pets-facial-expression-dataset", "format": "imagenet"},
    {
        "path": "./tunguz/1-million-fake-faces/1m_faces_00_01_02_03/1m_faces_00_01_02_03/1m_faces_00",
        "format": "image_dir",
    },
    {
        "path": "./gpiosenka/butterfly-images40-species",
        "format": "kaggle_image_csv",
        "ann_file": "./gpiosenka/butterfly-images40-species/butterflies and moths.csv",
        "columns": {"media": "filepaths", "label": "labels"},
    },
    {
        "path": "./meetnagadia/human-action-recognition-har-dataset/Human Action Recognition/train",
        "format": "kaggle_image_csv",
        "ann_file": "./meetnagadia/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv",
        "columns": {"media": "filename", "label": "label"},
    },
    {
        "path": "./antoreepjana/animals-detection-images-dataset",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./mahmoudreda55/satellite-image-classification/data", "format": "imagenet"},
    {"path": "./andrewmvd/medical-mnist", "format": "imagenet"},
    {"path": "./frabbisw/facial-age/face_age", "format": "imagenet"},
    {"path": "./odins0n/guie-custom-data/images_224", "format": "imagenet"},
    {
        "path": "./zippyz/cats-and-dogs-breeds-classification-oxford-dataset/images/images",
        "format": "kaggle_voc",
        "ann_path": "./zippyz/cats-and-dogs-breeds-classification-oxford-dataset/annotations/annotations/xmls",
    },
    {"path": "./muhammedtausif/best-selling-mobile-phones", "format": "tabular"},
    {
        "path": "./kneroma/tacotrashdataset/data",
        "format": "kaggle_image_csv",
        "ann_file": "./kneroma/tacotrashdataset/meta_df.csv",
        "columns": {
            "media": "img_file",
            "bbox": {
                "x1": "x",
                "y1": "y",
                "width": "width",
                "height": "height",
            },
            "label": "cat_name",
        },
    },
    {
        "path": "./vencerlanz09/bottle-synthetic-images-dataset/ImageClassesCombinedWithCOCOAnnotations/images_raw",
        "format": "kaggle_coco",
        "ann_file": "./vencerlanz09/bottle-synthetic-images-dataset/"
        + "ImageClassesCombinedWithCOCOAnnotations/coco_instances.json",
    },
    {
        "path": "./preetviradiya/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor",
        "format": "kaggle_image_csv",
        "ann_file": "./preetviradiya/brian-tumor-dataset/metadata.csv",
        "columns": {"media": "image", "label": "class"},
    },
    {
        "path": "./benjaminwarner/resized-2015-2019-blindness-detection-images/resized train 19",
        "format": "kaggle_image_csv",
        "ann_file": "./benjaminwarner/resized-2015-2019-blindness-detection-images/labels/trainLabels19.csv",
        "columns": {"media": "id_code", "label": "diagnosis"},
    },
    {"path": "./luisblanche/covidct", "format": "imagenet"},
    {"path": "./anujms/car-damage-detection/data1a/", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./iamsouravbanerjee/indian-actor-images-dataset/Bollywood Actor Images/Bollywood Actor Images",
        "format": "imagenet",
    },
    {"path": "./vikramtiwari/pix2pix-dataset/cityscapes/cityscapes/train", "format": "image_dir"},
    {"path": "./vencerlanz09/taco-dataset-yolo-format/", "format": "roboflow_yolo"},
    {
        "path": "./balraj98/deepglobe-road-extraction-dataset/train",
        "format": "kaggle_image_mask",
        "mask_path": "./balraj98/deepglobe-road-extraction-dataset/train",
    },
    {"path": "./kwentar/blur-dataset", "format": "imagenet"},
    {
        "path": "./vencerlanz09/plastic-paper-garbage-bag-synthetic-images/"
        + "ImageClassesCombinedWithCOCOAnnotations/images_raw",
        "format": "kaggle_coco",
        "ann_file": "./vencerlanz09/plastic-paper-garbage-bag-synthetic-images/"
        + "ImageClassesCombinedWithCOCOAnnotations/coco_instances.json",
    },
    {
        "path": "./utkarshsaxenadn/fast-food-classification-dataset/Fast Food Classification V2",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./robikscube/textocr-text-extraction-from-images-dataset/train_val_images/train_images",
        "format": "kaggle_image_csv",
        "ann_file": "./benjaminwarner/resized-2015-2019-blindness-detection-images/labels/trainLabels19.csv",
        "columns": {"media": "image_id", "bbox": "bbox"},
    },
    {
        "path": "./andrewmvd/helmet-detection/images",
        "format": "kaggle_voc",
        "ann_path": "./andrewmvd/helmet-detection/annotations/",
    },
    {
        "path": "./ashfakyeafi/road-vehicle-images-dataset/trafic_data/train/images",
        "format": "kaggle_yolo",
        "ann_path": "./ashfakyeafi/road-vehicle-images-dataset/trafic_data/train/labels",
    },
    {"path": "./mostafaabla/garbage-classification/garbage_classification", "format": "imagenet"},
    {
        "path": "./franciscoescobar/satellite-images-of-water-bodies/Water Bodies Dataset/Images",
        "format": "kaggle_image_mask",
        "mask_path": "./franciscoescobar/satellite-images-of-water-bodies/Water Bodies Dataset/Masks",
    },
    {"path": "./gavinarmstrong/open-sprayer-images/Docknet", "format": "imagenet_with_subset_dirs"},
    {"path": "./aaronyin/oneshotpokemon/kaggle-one-shot-pokemon/pokemon-a", "format": "image_dir"},
    {
        "path": "./arashnic/faces-age-detection-dataset/Train",
        "format": "kaggle_image_csv",
        "ann_file": "./arashnic/faces-age-detection-dataset/train.csv",
        "columns": {"media": "ID", "label": "Class"},
    },
    {"path": "./ayuraj/asl-dataset/asl_dataset", "format": "imagenet"},
    {
        "path": "./balabaskar/tom-and-jerry-image-classification/tom_and_jerry/tom_and_jerry",
        "format": "imagenet",
    },
    {
        "path": "./nafin59/monkeypox-skin-lesion-dataset/Original Images/Original Images",
        "format": "imagenet",
    },
    {"path": "./trolukovich/food11-image-dataset", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./mrgeislinger/asl-rgb-depth-fingerspelling-spelling-it-out/dataset5/A",
        "format": "imagenet",
    },
    {
        "path": "./balraj98/massachusetts-roads-dataset/tiff/test",
        "format": "kaggle_image_mask",
        "mask_path": "./balraj98/massachusetts-roads-dataset/tiff/test_labels",
    },
    {"path": "./kutaykutlu/forest-fire", "format": "imagenet"},
    {
        "path": "./sachinpatel21/pothole-image-dataset/pothole_image_data/Pothole_Image_Data",
        "format": "image_dir",
    },
    {
        "path": "./raddar/chest-xrays-indiana-university/images/images_normalized",
        "format": "kaggle_image_csv",
        "ann_file": "./raddar/chest-xrays-indiana-university/indiana_projections.csv",
        "columns": {"media": "filename", "label": "projection"},
    },
    {
        "path": "./ashfakyeafi/cat-dog-images-for-classification/cat_dog",
        "format": "kaggle_image_csv",
        "ann_file": "./ashfakyeafi/cat-dog-images-for-classification/cat_dog.csv",
        "columns": {"media": "image", "label": "labels"},
    },
    {
        "path": "./ashfakyeafi/cat-dog-images-for-classification/cat_dog",
        "format": "kaggle_image_csv",
        "ann_file": "./ashfakyeafi/cat-dog-images-for-classification/cat_dog.csv",
        "columns": {"media": "image", "label": "labels"},
    },
    {"path": "./meetnaren/goodreads-best-books/images/images", "format": "image_dir"},
    {"path": "./kondwani/eye-disease-dataset/Eye_diseases", "format": "imagenet"},
    {
        "path": "./niten19/face-shape-dataset/FaceShape Dataset",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./rishianand/devanagari-character-set/Images/Images", "format": "imagenet"},
    {"path": "./pkdarabi/cardetection", "format": "roboflow_yolo"},
    {
        "path": "./gpiosenka/cards-image-datasetclassification",
        "format": "kaggle_image_csv",
        "ann_file": "./gpiosenka/cards-image-datasetclassification/cards.csv",
        "columns": {"media": "filepaths", "label": "labels"},
    },
    {
        "path": "./piterfm/2022-ukraine-russia-war-equipment-losses-oryx/img_russia/img_russia",
        "format": "imagenet",
    },
    {
        "path": "./swaroopkml/cifar10-pngs-in-folders/cifar10/cifar10",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./tustunkok/synthetic-turkish-license-plates/license-plates", "format": "image_dir"},
    {
        "path": "./die9origephit/nike-adidas-and-converse-imaged",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./aniruddhsharma/structural-defects-network-concrete-crack-images/Pavements",
        "format": "imagenet",
    },
    {
        "path": "./andrewmvd/dog-and-cat-detection/images",
        "format": "kaggle_voc",
        "ann_path": "./andrewmvd/dog-and-cat-detection/annotations",
    },
    {
        "path": "./arpitjain007/game-of-deep-learning-ship-datasets/train/images",
        "format": "kaggle_image_csv",
        "ann_file": "./arpitjain007/game-of-deep-learning-ship-datasets/train/train.csv",
        "columns": {"media": "image", "label": "category"},
    },
    {
        "path": "./andrewmvd/tomato-detection/images",
        "format": "kaggle_voc",
        "ann_path": "./andrewmvd/tomato-detection/annotations",
    },
    {
        "path": "./sanikamal/rock-paper-scissors-dataset/Rock-Paper-Scissors",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./mdwaquarazam/agricultural-crops-image-classification/Agricultural-crops",
        "format": "imagenet",
    },
    {"path": "./akash2907/bird-species-classification/train_data/train_data", "format": "imagenet"},
    {
        "path": "./niteshfre/chessman-image-dataset/Chessman-image-dataset/Chess",
        "format": "imagenet",
    },
    {
        "path": "./niteshfre/chessman-image-dataset/Chessman-image-dataset/Chess",
        "format": "imagenet",
    },
    {"path": "./anshulmehtakaggl/chess-pieces-detection-images-dataset", "format": "imagenet"},
    {
        "path": "./usharengaraju/grassclover-dataset/biomass_data/train/images",
        "format": "kaggle_image_csv",
        "ann_file": "./usharengaraju/grassclover-dataset/biomass_data/train/biomass_train_data.csv",
        "columns": {"media": "image_file_name", "label": "label_type"},
    },
    {
        "path": "./farjanakabirsamanta/skin-cancer-dataset/Skin Cancer/Skin Cancer",
        "format": "kaggle_image_csv",
        "ann_file": "./farjanakabirsamanta/skin-cancer-dataset/HAM10000_metadata.csv",
        "columns": {"media": "image_id", "label": "dx"},
    },
    {"path": "./vaishnaviasonawane/indian-sign-language-dataset/data", "format": "imagenet"},
    {"path": "./l3llff/flowers/flowers", "format": "imagenet"},
    {"path": "./sharansmenon/animals141/dataset/dataset", "format": "imagenet"},
    {
        "path": "./balabaskar/wonders-of-the-world-image-classification/Wonders of World/Wonders of World",
        "format": "imagenet",
    },
    {"path": "./nancyalaswad90/us-airports", "format": "tabular"},
    {
        "path": "./wanderdust/skin-lesion-analysis-toward-melanoma-detection/skin-lesions",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./shashwatwork/knee-osteoarthritis-dataset-with-severity",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./sadhliroomyprime/spatial-vehicle-detection/images",
        "format": "kaggle_coco",
        "ann_file": "./sadhliroomyprime/spatial-vehicle-detection/COCO_Tokyo Japan.json",
    },
    {
        "path": "./sadhliroomyprime/spatial-vehicle-detection",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./linchundan/fundusimage1000/1000images", "format": "imagenet"},
    {
        "path": "./rashikrahmanpritom/plant-disease-recognition-dataset/Train/Train",
        "format": "imagenet",
    },
    {
        "path": "./ahemateja19bec1025/traffic-sign-dataset-classification/traffic_Data/DATA",
        "format": "imagenet",
    },
    {
        "path": "./maedemaftouni/large-covid19-ct-slice-dataset/curated_data/curated_data",
        "format": "imagenet",
    },
    {
        "path": "./lplenka/coco-car-damage-detection-dataset/train",
        "format": "kaggle_coco",
        "ann_file": "./lplenka/coco-car-damage-detection-dataset/train/COCO_train_annos.json",
    },
    {"path": "./rhtsingh/130k-images-512x512-universal-image-embeddings", "format": "imagenet"},
    {
        "path": "./anasmohammedtahir/covidqu/Lung Segmentation Data/Lung Segmentation Data/Train/COVID-19/images",
        "format": "kaggle_image_mask",
        "mask_path": "./anasmohammedtahir/covidqu/Lung Segmentation Data/"
        + "Lung Segmentation Data/Train/COVID-19/lung masks",
    },
    {"path": "./d4rklucif3r/cat-and-dogs/dataset/training_set/", "format": "imagenet"},
    {
        "path": "./vencerlanz09/shells-or-pebbles-an-image-classification-dataset",
        "format": "imagenet",
    },
    {
        "path": "./andrewmvd/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images",
        "format": "kaggle_image_mask",
        "mask_path": "./andrewmvd/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/mask",
    },
    {
        "path": "./williamscott701/memotion-dataset-7k/memotion_dataset_7k/images",
        "format": "kaggle_image_csv",
        "ann_file": "./williamscott701/memotion-dataset-7k/memotion_dataset_7k/labels.csv",
        "columns": {"media": "image_name", "label": "overall_sentiment"},
    },
    {"path": "./gunavenkatdoddi/eye-diseases-classification/dataset", "format": "imagenet"},
    {
        "path": "./soumendraprasad/fifa-2022-all-players-image-dataset/Images/Images/Group B/England Players",
        "format": "imagenet",
    },
    {"path": "./karimabdulnabi/fruit-classification10-class/MY_data/train", "format": "imagenet"},
    {"path": "./akhiljethwa/blood-cancer-image-dataset/Blood_Cancer", "format": "image_dir"},
    {
        "path": "./pkdarabi/the-drug-name-detection-dataset/train/images",
        "format": "kaggle_yolo",
        "ann_path": "./pkdarabi/the-drug-name-detection-dataset/train/labels",
    },
    {"path": "./dionisiusdh/indonesian-batik-motifs", "format": "imagenet"},
    {"path": "./subinium/emojiimage-dataset/image", "format": "imagenet"},
    {
        "path": "./sadhliroomyprime/motorcycle-night-ride-semantic-segmentation/"
        + "www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/images",
        "format": "kaggle_coco",
        "ann_file": "./sadhliroomyprime/motorcycle-night-ride-semantic-segmentation/"
        + "www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/COCO_motorcycle (pixel).json",
    },
    {
        "path": "./nitishabharathi/scene-classification/train-scene classification/train",
        "format": "kaggle_image_csv",
        "ann_file": "./nitishabharathi/scene-classification/train-scene classification/train.csv",
        "columns": {"media": "image_name", "label": "label"},
    },
    {"path": "./harshalhonde/starbucks-reviews-dataset", "format": "tabular"},
    {"path": "./eliasdabbas/emoji-data-descriptions-codepoints", "format": "tabular"},
    {
        "path": "./lgmoneda/br-coins/COCO_labelme_classification/classification/coco/JPEGImages",
        "format": "kaggle_coco",
        "ann_file": "./lgmoneda/br-coins/COCO_labelme_classification/classification/coco/annotations.json",
    },
    {
        "path": "./preetviradiya/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset",
        "format": "imagenet",
    },
    {
        "path": "./tapakah68/yandextoloka-water-meters-dataset/WaterMeters/images",
        "format": "kaggle_image_mask",
        "mask_path": "./tapakah68/yandextoloka-water-meters-dataset/WaterMeters/masks",
    },
    {"path": "./yusufberksardoan/traffic-detection-project", "format": "roboflow_yolo"},
    {
        "path": "./jtiptj/chest-xray-pneumoniacovid19tuberculosis",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./vijaykumar1799/face-mask-detection/Dataset", "format": "imagenet"},
    {"path": "./nancyalaswad90/trip-data-ford-go-bike", "format": "tabular"},
    {"path": "./alifrahman/covid19-chest-xray-image-dataset/dataset", "format": "imagenet"},
    {"path": "./vitaminc/cigarette-smoker-detection/data/data", "format": "imagenet"},
    {
        "path": "./tapakah68/segmentation-full-body-mads-dataset/"
        + "segmentation_full_body_mads_dataset_1192_img/segmentation_full_body_mads_dataset_1192_img/images",
        "format": "kaggle_image_mask",
        "mask_path": "./tapakah68/segmentation-full-body-mads-dataset/"
        + "segmentation_full_body_mads_dataset_1192_img/segmentation_full_body_mads_dataset_1192_img/masks",
    },
    {"path": "./mpwolke/cusersmarildownloadsdeadjpg", "format": "image_dir"},
    {"path": "./lsind18/gemstones-images", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./balabaskar/golden-foot-football-players-image-dataset/football_golden_foot/football_golden_foot",
        "format": "imagenet",
    },
    {
        "path": "./fantacher/neu-metal-surface-defects-data/NEU Metal Surface Defects Data",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./olgabelitskaya/style-color-images/style",
        "format": "kaggle_image_csv",
        "ann_file": "./olgabelitskaya/style-color-images/style/style.csv",
        "columns": {"media": "file", "label": "brand_name"},
    },
    {"path": "./salmaneunus/rock-classification/Dataset/Metamorphic", "format": "imagenet"},
    {"path": "./adhoppin/blood-cell-detection-datatset", "format": "roboflow_yolo"},
    {"path": "./carlosrunner/pizza-not-pizza/pizza_not_pizza", "format": "imagenet"},
    {"path": "./mpwolke/cusersmarildownloadsdeadjpg", "format": "image_dir"},
    {
        "path": "./maciejgronczynski/biggest-genderface-recognition-dataset/faces",
        "format": "imagenet",
    },
    {
        "path": "./balabaskar/cricket-legends-in-world-image-dataset/Cricket Legends/Cricket Legends",
        "format": "imagenet",
    },
    {
        "path": "./mksaad/wider-face-a-face-detection-benchmark/WIDER_train/WIDER_train/images",
        "format": "imagenet",
    },
    {
        "path": "./vencerlanz09/insect-village-synthetic-dataset/ImageClassesCombinedWithCOCOAnnotations/images_raw",
        "format": "kaggle_coco",
        "ann_file": "./vencerlanz09/insect-village-synthetic-dataset/"
        + "ImageClassesCombinedWithCOCOAnnotations/coco_instances.json",
    },
    {
        "path": "./balraj98/massachusetts-buildings-dataset/png/train",
        "format": "kaggle_image_mask",
        "mask_path": "./balraj98/massachusetts-buildings-dataset/png/train_labels",
    },
    {"path": "./wwymak/architecture-dataset/arcDataset", "format": "imagenet"},
    {"path": "./kmader/parkinsons-drawings/drawings/spiral", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./ivanfel/honey-bee-pollen/PollenDataset/images",
        "format": "kaggle_image_csv",
        "ann_file": "./ivanfel/honey-bee-pollen/PollenDataset/pollen_data.csv",
        "columns": {"media": "filename", "label": "pollen_carrying"},
    },
    {
        "path": "./birdy654/cifake-real-and-ai-generated-synthetic-images",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./theoviel/rsna-2023-abdominal-trauma-detection-pngs-18", "format": "image_dir"},
    {"path": "./coloradokb/dandelionimages/Images", "format": "imagenet"},
    {"path": "./ayanzadeh93/color-classification/ColorClassification", "format": "imagenet"},
    {"path": "./vijaygiitk/multiclass-weather-dataset/dataset", "format": "imagenet"},
    {"path": "./ashfakyeafi/glasses-classification-dataset", "format": "imagenet_with_subset_dirs"},
    {
        "path": "./samuelcortinhas/muffin-vs-chihuahua-image-classification",
        "format": "imagenet_with_subset_dirs",
    },
    {
        "path": "./subinium/highresolution-anime-face-dataset-512x512/portraits",
        "format": "image_dir",
    },
    {"path": "./edoardoba/fitness-exercises-with-animations", "format": "tabular"},
    {
        "path": "./vishesh1412/celebrity-face-image-dataset/Celebrity Faces Dataset",
        "format": "imagenet",
    },
    {
        "path": "./mcagriaksoy/amateur-unmanned-air-vehicle-detection-dataset/Database1/Database1",
        "format": "kaggle_yolo",
        "ann_path": "./mcagriaksoy/amateur-unmanned-air-vehicle-detection-dataset/Database1/Database1",
    },
    {
        "path": "./dataclusterlabs/fire-and-smoke-dataset/"
        + "Datacluster Fire and Smoke Sample/Datacluster Fire and Smoke Sample",
        "format": "kaggle_voc",
        "ann_path": "./dataclusterlabs/fire-and-smoke-dataset/Annotations/Annotations",
    },
    {
        "path": "./vencerlanz09/pharmaceutical-drugs-and-vitamins-dataset-v2/Capsure Dataset",
        "format": "imagenet_with_subset_dirs",
    },
    {"path": "./fernando2rad/brain-tumor-mri-images-44c", "format": "imagenet"},
    {
        "path": "./roydatascience/training-car/data",
        "format": "kaggle_image_csv",
        "ann_file": "./roydatascience/training-car/data/driving_log.csv",
        "columns": {"media": "center", "label": "brake"},
    },
    {"path": "./stoicstatic/face-recognition-dataset/Face Data/Face Dataset", "format": "imagenet"},
]

for kwargs in datasets:
    dataset = dm.Dataset.import_from(**kwargs)

265
